Guillarte, Dana Louise A.

DATA100 - S17

# Zomato Exercise

Zomato is a restaurant aggregator website. The platform provides information, menus and user-reviews of restaurants as well as food delivery options from partner restaurants in select cities. You have been provided 3 datasets:

- zomato.csv contains restaurant information like average_cost_for_two, cuisines
- location.csv contains location of the restaurants like address, city, locality & etc
- ratings.csv contains aggregate rating for each restaurant

The different files are connected using the `id` columns which can be found in the different files. The id column is a unique identifier to represent a restaurant. You can use this to **merge** the different files.

In [1]:
import pandas as pd
import numpy as np

In [4]:
location_addr = 'https://raw.githubusercontent.com/DaLouuu/Zomato-Exercise/main/location.csv'
location_df = pd.read_csv(location_addr)

zomato_addr = 'https://raw.githubusercontent.com/DaLouuu/Zomato-Exercise/main/zomato.csv'
zomato_df = pd.read_csv(zomato_addr)

ratings_addr = 'https://raw.githubusercontent.com/DaLouuu/Zomato-Exercise/main/ratings.csv'
ratings_df = pd.read_csv(ratings_addr)

In [5]:
location_df.head()

zomato_df.head()

ratings_df.head()

,id,aggregate_rating,rating_color,rating_text,votes
0,6316125,4.9,3F7E00,Excellent,1394
1,18409457,4.6,3F7E00,Excellent,602
2,6307689,4.0,5BA829,Very Good,563
3,18581637,4.0,5BA829,Very Good,413
4,6315438,4.6,3F7E00,Excellent,960


1. Merge the zomato, location & ratings file into a signle dataframe and store it to the `merged_df` variable **(3pts)**

In [7]:
merged_df = zomato_df.merge(location_df, on='id', how='inner')
merged_df = merged_df.merge(ratings_df, on='id', how='inner')

print(merged_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6830 entries, 0 to 6829
Data columns (total 35 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              6830 non-null   int64  
 1   average_cost_for_two            6830 non-null   int64  
 2   book_url                        518 non-null    object 
 3   cuisines                        6829 non-null   object 
 4   currency                        6830 non-null   object 
 5   deeplink                        6830 non-null   object 
 6   events_url                      6830 non-null   object 
 7   featured_image                  6248 non-null   object 
 8   has_online_delivery             6830 non-null   int64  
 9   has_table_booking               6830 non-null   int64  
 10  include_bogo_offers             6830 non-null   bool   
 11  is_book_form_web_view           6828 non-null   float64
 12  is_delivering_now               68

In [8]:
assert (6830, 35)==merged_df.shape

cols = ['id', 'address', 'city', 'city_id', 'country_id', 'latitude',
       'locality', 'locality_verbose', 'longitude', 'zipcode',
       'aggregate_rating', 'rating_color', 'rating_text', 'votes',
       'average_cost_for_two', 'book_url', 'cuisines', 'currency', 'deeplink',
       'events_url', 'featured_image', 'has_online_delivery',
       'has_table_booking', 'include_bogo_offers', 'is_book_form_web_view',
       'is_delivering_now', 'is_table_reservation_supported',
       'is_zomato_book_res', 'menu_url', 'mezzo_provider', 'name',
       'photos_url', 'price_range', 'thumb', 'url']

assert sorted(merged_df.columns) == sorted(cols)

2. Create a function that returns the average aggregate rating for each city. Expected returned value is a Series datatype which contains name of city as index and average aggregate rating as value. **(2pts)**

In [9]:
def exercise_2(merged_df):
    avg_rating_per_city = merged_df.groupby('city')['aggregate_rating'].mean()

    return avg_rating_per_city

In [10]:
res_series = exercise_2(merged_df)
assert True == np.isclose(3.302075, res_series[res_series.index == 'Pasay City'],
                          rtol=1e-05, atol=1e-08, equal_nan=False)[0]
assert True == np.isclose(1.890741, res_series[res_series.index == 'Valenzuela City'],
                          rtol=1e-05, atol=1e-08, equal_nan=False)[0]
assert True == np.isclose(1.558333, res_series[res_series.index == 'Malabon City'],
                          rtol=1e-05, atol=1e-08, equal_nan=False)[0]

3. Using the apply function of pandas convert the rating_text to a numeric equivalent. You are to introduce a new column in merged_df called `rating_numeric` which is based on the value from rating_text: **(3pts)**

Excellent = 5 <br>
Very Good = 4 <br>
Good = 3 <br>
Average = 2 <br>
Poor = 1 <br>
Not rated = 0 <br>

In [13]:
def convert_rating_text_to_numeric(rating_text):
    rating_map = {
        'Excellent': 5,
        'Very Good': 4,
        'Good': 3,
        'Average': 2,
        'Poor': 1,
        'Not rated': 0
    }

    return rating_map.get(rating_text, 0)

merged_df['rating_numeric'] = merged_df['rating_text'].apply(convert_rating_text_to_numeric)

In [12]:
assert 868 == merged_df[merged_df.rating_numeric == 0].shape[0]
assert True == ('rating_numeric' in merged_df.columns)

4. Create a function which filters the merged dataframe to only contain rows coming in from a specific city. Expected returned value is a dataframe **(2pts)**

In [14]:
def exercise_4(merged_df, city):
    filtered_df = merged_df[merged_df['city'] == city]

    return filtered_df

In [15]:
assert 'Makati City' == exercise_4(merged_df, 'Makati City').city.unique()[0]
assert (976, 36) == exercise_4(merged_df, 'Makati City').shape

assert 'Mandaluyong City' == exercise_4(merged_df, 'Mandaluyong City').city.unique()[0]
assert (443, 36) == exercise_4(merged_df, 'Mandaluyong City').shape

assert (0, 36) == exercise_4(merged_df, 'Quezon  City').shape

5. Create a function to filter the dataframe to only contain certain columns passed in the function. Expected returned value is a dataframe **(1pt)**

In [16]:
def exercise_5(merged_df, cols_to_filter):
    filtered_df = merged_df[cols_to_filter]

    return filtered_df

In [18]:
cols = ['name', 'cuisines', 'city', 'aggregate_rating']
assert sorted(cols) == sorted(exercise_5(merged_df, cols).columns)
assert (6830, 4) == exercise_5(merged_df, cols).shape

cols = ['average_cost_for_two', 'book_url', 'cuisines', 'currency', 'deeplink',
       'events_url', 'featured_image', 'has_online_delivery',
       'has_table_booking', 'include_bogo_offers', 'is_book_form_web_view']
assert sorted(cols) == sorted(exercise_5(merged_df, cols).columns)
assert (6830, 11) == exercise_5(merged_df, cols).shape

6.  What is the most common restaurant name in the dataset? **(2pts)** Write down your answer and show code used to come up with the answer.

In [19]:
restaurant_counts = merged_df['name'].value_counts()

most_common_name = restaurant_counts.idxmax()

count = restaurant_counts[most_common_name]

print(f"The most common restaurant name is '{most_common_name}' with {count} occurrences.")

The most common restaurant name is 'Starbucks' with 99 occurrences.


7. Which restaurant has the most expensive average cost for two? **(2pts)** Write down your answer and show code used to come up with the answer.

In [20]:
most_expensive_restaurant = merged_df.sort_values(by='average_cost_for_two', ascending=False).iloc[0]

restaurant_name = most_expensive_restaurant['name']
cost_for_two = most_expensive_restaurant['average_cost_for_two']

print(f"The most expensive restaurant is '{restaurant_name}' with an average cost for two of {cost_for_two}.")

The most expensive restaurant is 'The Tasting Room' with an average cost for two of 10000.


8. How many restaurants do not have a zipcode? **(1pt)** Write down your answer and show code used to come up with the answer.

In [21]:
restaurants_without_zipcode = merged_df['zipcode'].isna().sum()

print(f"There are {restaurants_without_zipcode} restaurants that do not have a zipcode.")

There are 6713 restaurants that do not have a zipcode.


9. How many restaurants in the dataset allow online delivery? **(1pt)** Write down your answer and show code used to come up with the answer.

In [22]:
restaurants_with_online_delivery = merged_df[merged_df['has_online_delivery'] == 1].shape[0]

print(f"There are {restaurants_with_online_delivery} restaurants that allow online delivery.")

There are 0 restaurants that allow online delivery.


10. Which locatlity has more expensive food? Poblacion or Salcedo Village? **(3pts)** Write down your answer and show code used to come up with the answer.


In [25]:
poblacion_cost = merged_df[merged_df['locality'] == 'Poblacion']['average_cost_for_two'].mean()
salcedo_cost = merged_df[merged_df['locality'] == 'Salcedo Village']['average_cost_for_two'].mean()

if poblacion_cost > salcedo_cost:
    more_expensive_locality = 'Poblacion'
    cost = poblacion_cost
else:
    more_expensive_locality = 'Salcedo Village'
    cost = salcedo_cost

print(f"{more_expensive_locality} has more expensive food with an average cost for two of {cost:.2f}.")

Salcedo Village has more expensive food with an average cost for two of 989.10.
